In [1]:
import numpy as np
import sys
from IPython.display import Image

In [2]:
def GRAYencoder(x):
	for idx in range(len(x)):
		if idx == 0:
			y = x[idx]
		else:
			y = y + str ( int( x[idx] ) ^ int( x[idx-1] ) )
	return y

In [3]:
def PSKmodulator(modulation_order, phase_shift = np.pi/4, symbol_mapping = 'Gray', input_type = 'Binary'):
    s = [0+i for i in range(modulation_order)]
    m = list(np.exp(1j*phase_shift + 1j*2*np.pi*np.array(s)/modulation_order))
    dict_out = {}
    if input_type != 'Binary' and input_type != 'Decimal':
        print("Wrong input data type (should be 'Decimal' or 'Binary'). Now input_type = " + str(input_type))
        sys.exit(0) 
    elif input_type == 'Decimal':
        if symbol_mapping != 'Binary' and symbol_mapping != 'Gray':
            print("Wrong mapping rule (should be 'Gray' or 'Binary'). Now symbol_mapping = " + str(symbol_mapping))
            sys.exit(0)
        elif symbol_mapping == 'Gray':
            s2 = []
            for i in s:
                symbol = bin(i)[2:]
                if len(symbol) < np.log2(modulation_order):
                    symbol = int( (np.log2(modulation_order) - len(symbol)) )*'0'+symbol
                s2.append( int(GRAYencoder(symbol), 2 ) )
            s = s2
    elif input_type == 'Binary':
        if symbol_mapping == 'Binary':
            b = []
            for i in s:
                a = bin(i)[2:]
                if len(a) < np.log2(modulation_order):
                    a = int((np.log2(modulation_order) - len(a)))*'0'+a
                if np.log2(modulation_order)%2 == 0:
                    a = a[::-1]
                b.append(a)
            s = b
        elif symbol_mapping == 'Gray':
            s2 = []
            for i in s:
                symbol = bin(i)[2:]
                if len(symbol) < np.log2(modulation_order):
                    symbol = int( (np.log2(modulation_order) - len(symbol)) )*'0'+symbol
                s2.append(GRAYencoder(symbol))
            s = []
            for i in s2:
                if np.log2(modulation_order)%2 == 0:
                    i = i[::-1]
                s.append(i)
        else:
            print("Wrong mapping rule (should be 'Gray' or 'Binary'). Now input_type = " + str(symbol_mapping))
            sys.exit(0)
    for x, y in zip(s, m):
        dict_out[x] = y
    return dict_out

## Binary input

In [4]:
PSKmodulator(4)

{'00': (0.7071067811865476+0.7071067811865476j),
 '01': (0.7071067811865474-0.7071067811865477j),
 '10': (-0.7071067811865475+0.7071067811865476j),
 '11': (-0.7071067811865477-0.7071067811865475j)}

In [5]:
PSKmodulator(4, symbol_mapping = 'Binary')

{'00': (0.7071067811865476+0.7071067811865476j),
 '01': (-0.7071067811865477-0.7071067811865475j),
 '10': (-0.7071067811865475+0.7071067811865476j),
 '11': (0.7071067811865474-0.7071067811865477j)}

In [6]:
PSKmodulator(4, 0)

{'00': (1+0j),
 '01': (-1.8369701987210297e-16-1j),
 '10': (6.123233995736766e-17+1j),
 '11': (-1+1.2246467991473532e-16j)}

## Decimal input

In [7]:
Image(url= "https://www.mathworks.com/help/comm/ug/qpsk_const_diagram_gray.png" )

(used for verification)

In [8]:
PSKmodulator(4, input_type = 'Decimal')

{0: (0.7071067811865476+0.7071067811865476j),
 1: (-0.7071067811865475+0.7071067811865476j),
 2: (0.7071067811865474-0.7071067811865477j),
 3: (-0.7071067811865477-0.7071067811865475j)}

In [9]:
Image(url= "https://www.mathworks.com/help/comm/ug/qpsk_const_diagram_binary.png")

(used for verification)

In [10]:
PSKmodulator(4, symbol_mapping = 'Binary', input_type = 'Decimal')

{0: (0.7071067811865476+0.7071067811865476j),
 1: (-0.7071067811865475+0.7071067811865476j),
 2: (-0.7071067811865477-0.7071067811865475j),
 3: (0.7071067811865474-0.7071067811865477j)}

## 8-PSK

In [11]:
Image(url= "https://www.mathworks.com/help/comm/ug/psk8_const_diagram_gray.png")

In [12]:
PSKmodulator(8, np.pi/6)

{'000': (0.8660254037844387+0.49999999999999994j),
 '001': (0.25881904510252096+0.9659258262890682j),
 '010': (-0.9659258262890682+0.258819045102521j),
 '011': (-0.4999999999999998+0.8660254037844387j),
 '100': (0.9659258262890683-0.2588190451025207j),
 '101': (0.5000000000000001-0.8660254037844386j),
 '110': (-0.8660254037844388-0.4999999999999997j),
 '111': (-0.25881904510252063-0.9659258262890683j)}

## Exceptions (errors)

In [13]:
#PSKmodulator(4, input_type = 'Decimal1')

In [14]:
#PSKmodulator(4, symbol_mapping = 'Binary1')

In [15]:
#PSKmodulator(4, symbol_mapping = 'Binary1', input_type = 'Decimal')

# Demodulator

In [16]:
def PSKDemodulator(modulation_order, phase_shift = np.pi/4, symbol_mapping = 'Gray'):
	zeros = []
	ones = []
	for c in range(int(np.log2(modulation_order))):
		zeros.append([])
		ones.append([])
	codebook = PSKmodulator(modulation_order, phase_shift, symbol_mapping, 'Decimal')
	s = [i for i in range(modulation_order)]
	b = []
	for i in s:
		a = bin(i)[2:]
		if len(a) < np.log2(modulation_order):
			a = int((np.log2(modulation_order) - len(a)))*'0'+a
		if np.log2(modulation_order)%2 == 0:
			a = a[::-1]
		b.append(a)
	for idx, n in enumerate(b):
		for ind, m in enumerate(n):
			if m == '0':
				zeros[ind].append(codebook[idx])
			else:
				ones[ind].append(codebook[idx])
	return zeros, ones

## QPSK (pi/4  phase rotation)

In [17]:
zeros, ones = PSKDemodulator(4)

In [18]:
print('Zeros on the first position can be: '+str(zeros[0]))
print('Ones on the first position can be: '+str(ones[0]))
print('Zeros on the second position can be: '+str(zeros[1]))
print('Ones on the second position can be: '+str(ones[1]))

Zeros on the first position can be: [(0.7071067811865476+0.7071067811865476j), (0.7071067811865474-0.7071067811865477j)]
Ones on the first position can be: [(-0.7071067811865475+0.7071067811865476j), (-0.7071067811865477-0.7071067811865475j)]
Zeros on the second position can be: [(0.7071067811865476+0.7071067811865476j), (-0.7071067811865475+0.7071067811865476j)]
Ones on the second position can be: [(0.7071067811865474-0.7071067811865477j), (-0.7071067811865477-0.7071067811865475j)]


## 8-PSK

In [19]:
zeros, ones = PSKDemodulator(8, np.pi/6)

In [20]:
print('Zeros on the first position can be: '+str(zeros[0]))
print('')
print('Ones on the first position can be: '+str(ones[0]))
print('')
print('Zeros on the second position can be: '+str(zeros[1]))
print('')
print('Ones on the second position can be: '+str(ones[1]))
print('')
print('Zeros on the third position can be: '+str(zeros[2]))
print('')
print('Ones on the third position can be: '+str(ones[2]))
print('')

Zeros on the first position can be: [(0.8660254037844387+0.49999999999999994j), (0.25881904510252096+0.9659258262890682j), (-0.9659258262890682+0.258819045102521j), (-0.4999999999999998+0.8660254037844387j)]

Ones on the first position can be: [(0.9659258262890683-0.2588190451025207j), (0.5000000000000001-0.8660254037844386j), (-0.8660254037844388-0.4999999999999997j), (-0.25881904510252063-0.9659258262890683j)]

Zeros on the second position can be: [(0.8660254037844387+0.49999999999999994j), (0.25881904510252096+0.9659258262890682j), (0.9659258262890683-0.2588190451025207j), (0.5000000000000001-0.8660254037844386j)]

Ones on the second position can be: [(-0.9659258262890682+0.258819045102521j), (-0.4999999999999998+0.8660254037844387j), (-0.8660254037844388-0.4999999999999997j), (-0.25881904510252063-0.9659258262890683j)]

Zeros on the third position can be: [(0.8660254037844387+0.49999999999999994j), (-0.9659258262890682+0.258819045102521j), (0.9659258262890683-0.2588190451025207j), 